In [1]:
using Fermionic
using SparseArrays
using LinearAlgebra

# Fermionic Dataset

### Funciones complementarias
filter_lvl recoge solo los niveles que se corresponden con niveles totalmente ocupados (los no nulos en el fundamental).
map_to_2n lleva estos niveles a la base de 2^n del sistema de n spines, donde acá n va a coincidir con el numero de niveles (en un sistema de d=8, tengo 4 niveles degenerados. Eso mapea a un sistema de 4 spines)

In [84]:
function map_to_2n(estado, index, d)
    l = size(estado)[1]
    #lo puedo hacer sparse si prefieren, es mas lindo pero quizas no le gusta a initialize del qiskit
    estado2n = zeros(l,2^d)
    index2 = index[1:size(estado)[2]]
    for j in 1:l
        for (i,v) in enumerate(index2)
            estado2n[j,Int(v+1)]=estado[j,i]
        end
    end
    return estado2n    
end

function filter_lvl(b)
    bin, d = size(b)
    f = [sum([b[t,k]*b[t,k+1] for k in 1:2:d]) for t in 1:bin]
    return f
end
;

### Simulación
Simulamos el Hamiltoniano exacto para cada acoplamiento y lo guardamos junto a las etiquetas de superconductividad definidas por BCS.

In [90]:
d = 8 #Es un sistema de 4 niveles con degeneración 2
nume = Int(d/2)

e0 = 1.0
gmin = 0.0
gmax = 5.0
step = 0.01

b, index = basis_m(d,nume)

epsilon = [e0*(i-d/4-1/2) for i in 1:d/2]
gc = 1/sum(1 ./abs.(2*epsilon))
epsilon = sort([epsilon; epsilon])
h0 = sum([epsilon[i]*(cdc(b,index,i,i) + cdc(b,index,i+1,i+1)) for i in 1:2:(Int(d)-1)])
hi = sum([sum([if i==j spzeros(binomial(d,nume), binomial(d,nume)) else -(cdc(b,index,j,i+1)*cdc(b,index,j+1,i)) end
                    for i in 1:2:(Int(d)-1)]) for j in 1:2:(Int(d)-1)])

g = zeros(0)
#si quieren lo puedo hacer sparse poniendo spzeros aca abajo
estado = zeros(Int((gmax-gmin)/step+1),binomial(d,nume))
labels = zeros(0)
count = 1
for k in gmin:step:gmax
    append!(g,round(k, digits=10))
    h = h0 - (round(k, digits=10))*hi
    fundamental = eigvecs(Matrix(h))[:,1];
    fundamental = fundamental/sqrt(fundamental'*fundamental)
    fundamental = [round(fundamental[i], digits=15) for i in 1:binomial(d,nume)]
    estado[count,:] = fundamental
    if k>=gc
        lab=1
    else
        lab=0
    end
    append!(labels,lab)
    count = count+1
end

El gc (acoplamiento crítico de bcs) fue chequeado con mathematica

In [91]:
gc

0.375

### Traducción a qiskit y guardado
Hacemos la conversión a la base de qiskit. Es decir base de 2^n con n el num de niveles, que serán los n qubits del input en qiskit.

In [92]:
a = filter_lvl(b)
posiciones = findall(x->x==2, a)
e = estado[:,posiciones]
b2,i2=basis_m(Int(d/2),Int(nume/2))

estado2n=map_to_2n(e, i2, Int(d/2))

501×16 Array{Float64,2}:
 0.0  0.0  0.0  0.0          0.0  0.0         …  0.0  1.0       0.0  0.0  0.0
 0.0  0.0  0.0  1.46663e-5   0.0  0.00167501     0.0  0.99998   0.0  0.0  0.0
 0.0  0.0  0.0  5.89933e-5   0.0  0.00336677     0.0  0.999918  0.0  0.0  0.0
 0.0  0.0  0.0  0.000133465  0.0  0.00507533     0.0  0.999813  0.0  0.0  0.0
 0.0  0.0  0.0  0.000238553  0.0  0.00680073     0.0  0.999664  0.0  0.0  0.0
 0.0  0.0  0.0  0.000374715  0.0  0.00854301  …  0.0  0.999469  0.0  0.0  0.0
 0.0  0.0  0.0  0.000542392  0.0  0.0103022      0.0  0.999227  0.0  0.0  0.0
 0.0  0.0  0.0  0.000742009  0.0  0.0120782      0.0  0.998937  0.0  0.0  0.0
 0.0  0.0  0.0  0.000973971  0.0  0.0138711      0.0  0.998598  0.0  0.0  0.0
 0.0  0.0  0.0  0.00123866   0.0  0.0156808      0.0  0.998207  0.0  0.0  0.0
 0.0  0.0  0.0  0.00153645   0.0  0.0175072   …  0.0  0.997765  0.0  0.0  0.0
 0.0  0.0  0.0  0.00186767   0.0  0.0193503      0.0  0.99727   0.0  0.0  0.0
 0.0  0.0  0.0  0.00223263   0.0  0.021

Agreguemosle acoplamientos y labeles a los datos

In [93]:
estado_l = hcat(g,estado2n,labels)

501×18 Array{Float64,2}:
 0.0   0.0  0.0  0.0  0.0          0.0  …  0.0  1.0       0.0  0.0  0.0  0.0
 0.01  0.0  0.0  0.0  1.46663e-5   0.0     0.0  0.99998   0.0  0.0  0.0  0.0
 0.02  0.0  0.0  0.0  5.89933e-5   0.0     0.0  0.999918  0.0  0.0  0.0  0.0
 0.03  0.0  0.0  0.0  0.000133465  0.0     0.0  0.999813  0.0  0.0  0.0  0.0
 0.04  0.0  0.0  0.0  0.000238553  0.0     0.0  0.999664  0.0  0.0  0.0  0.0
 0.05  0.0  0.0  0.0  0.000374715  0.0  …  0.0  0.999469  0.0  0.0  0.0  0.0
 0.06  0.0  0.0  0.0  0.000542392  0.0     0.0  0.999227  0.0  0.0  0.0  0.0
 0.07  0.0  0.0  0.0  0.000742009  0.0     0.0  0.998937  0.0  0.0  0.0  0.0
 0.08  0.0  0.0  0.0  0.000973971  0.0     0.0  0.998598  0.0  0.0  0.0  0.0
 0.09  0.0  0.0  0.0  0.00123866   0.0     0.0  0.998207  0.0  0.0  0.0  0.0
 0.1   0.0  0.0  0.0  0.00153645   0.0  …  0.0  0.997765  0.0  0.0  0.0  0.0
 0.11  0.0  0.0  0.0  0.00186767   0.0     0.0  0.99727   0.0  0.0  0.0  0.0
 0.12  0.0  0.0  0.0  0.00223263   0.0     0.0  0.9

Guardemoslos en formato csv:

In [101]:
header2n=String[]
push!(header2n,"g")
for i in 0:(2^Int(d/2)-1)
    binary_base = bitstring(i)[65-Int(d/2):64]
    push!(header2n,binary_base)
end
push!(header2n,"label")

CSV.write("estados_super2.csv",  DataFrame(estado_l), header=header2n);

### Chequeos
Esto de abajo son chequeas con métodos alternativos usando simetrías. Está dando igual.

In [13]:
# Con simetrias

d=4
nume = Int(d/2)

o = Op(d)

e0 = 1.0
epsilon = [2*e0*(i-d/2-1/2) for i in 1:d]
g = 5.0
h0 = spzeros(binomial(d,nume),binomial(d,nume))
bam,_ = basis_m(d,nume)

for i in 1:binomial(d,nume)
    h0[i,i] = epsilon'*bam[i,:]
end

hi = spzeros(binomial(d,nume),binomial(d,nume))
for i in 1:binomial(d,nume)
    for j in 1:binomial(d,nume)
        if length(findall(in(indx(bam[i,:])), indx(bam[j,:]))) == nume - 1
            hi[i,j] = g
        end
    end
end

h = h0 - hi;
fundamental = eigvecs(Matrix(h))[:,1]


6-element Array{Float64,1}:
 -0.3302502483797297
 -0.36343295217043653
 -0.4004693453872221
 -0.40046934538722206
 -0.442734717468786
 -0.4920221940928091

In [95]:
# Con Fermionic 


d = 8 #this will be a system of 2 levels
nume = Int(d/2)

e0 = 1.0
gmin = 5.0
gmax = 5.0
step = 0.01

b, index = basis_m(d,nume)

epsilon = [e0*(i-d/4-1/2) for i in 1:d/2]
gc = 1/sum(1 ./abs.(2*epsilon))
epsilon = sort([epsilon; epsilon])
h0 = sum([epsilon[i]*(cdc(b,index,i,i) + cdc(b,index,i+1,i+1)) for i in 1:2:(Int(d)-1)])
hi = sum([sum([if i==j spzeros(binomial(d,nume), binomial(d,nume)) else -(cdc(b,index,j,i+1)*cdc(b,index,j+1,i)) end
                    for i in 1:2:(Int(d)-1)]) for j in 1:2:(Int(d)-1)])

g = zeros(0)
#si quieren lo puedo hacer sparse poniendo spzeros aca abajo
estado = zeros(Int((gmax-gmin)/step+1),binomial(d,nume))
labels = zeros(0)
count = 1
for k in gmin:step:gmax
    append!(g,round(k, digits=10))
    h = h0 - (round(k, digits=10))*hi
    fundamental = eigvecs(Matrix(h))[:,1];
    fundamental = fundamental/sqrt(fundamental'*fundamental)
    fundamental = [round(fundamental[i], digits=15) for i in 1:binomial(d,nume)]
    estado[count,:] = fundamental
    if k>=gc
        lab=1
    else
        lab=0
    end
    append!(labels,lab)
    count = count+1
end

In [97]:
a = filter_lvl(b)
posiciones = findall(x->x==2, a)
e = estado[:,posiciones]
b2,i2=basis_m(Int(d/2),Int(nume/2))

e

1×6 Array{Float64,2}:
 0.33025  0.363433  0.400469  0.400469  0.442735  0.492022

Da igual a menos de una fase global (no relevante)